## Verification program
by [Lei You](https://leiyou.me/)

In [102]:
import numpy as np
import pandas as pd
import scipy.io
import math
from ast import literal_eval

In [103]:
def dbm_to_mw(x):
    return 10**(x/10)

def w(j, load):
    i = cell_of_ues[j]
    gain_ij = gain_mat[i][j]
    inter_cell_interf_j = np.dot(power*load, gain_mat[:,j]) - power[i]*load[i]*gain_ij
    w_j = (inter_cell_interf_j + noise) / gain_ij
    
    return w_j


def sinr_oma(j, load):
    i = cell_of_ues[j]
    w_j = w(j, load)
    return power[i] / w_j

def sinr_noma(j, h, q_j, q_h, load):
    w_j = w(j, load)
    w_h = w(h, load)
    
    theta_hj = 1 if w_h<=w_j else 0
    if w_h==w_j:
        theta_hj = 1 if j<h else 0
    
    sinr = q_j / (q_h * theta_hj + w_j)
    return sinr

def demand_oma(j, x_j, load):
    """
         j: index of UE
       x_j: resource allocated to UE j for doing OMA
      load: vector of converged network load
    """
    return x_j * math.log2(1+sinr_oma(j,load))

def demand_noma(j, h, q_j, q_h, x_u, load):
    """
        j: index of UE
        h: index of UE 
      x_u: resource allocated to pair(j,h) for doing NOMA
     load: vector of converged network load
      q_j: power split of UE j
      q_h: power split of UE h
    """
    return x_u * math.log2(1+sinr_noma(j, h, q_j, q_h, load))


In [104]:
# Parameter settings
gain_mat_dbm = scipy.io.loadmat('Gain_7-2-30.mat')['gain']
gain_mat = np.array(list(map(dbm_to_mw, gain_mat_dbm)))

N_CELL, N_UE = gain_mat_dbm.shape

cell_list = range(N_CELL)
ue_list = range(N_UE)

# sigma
sigma = dbm_to_mw(-174)

# bandwidth on one RU
bandwidth = 180000

# noise
noise = sigma * bandwidth

# normalized demand
demand = 0.0967
demand_list = np.array([demand for j in range(N_UE)])

# association
cell_of_ues = list(map(int,scipy.io.loadmat('cell_of_ues.mat')['association'][0]))

# cell power
N_MACRO = 7
N_SMALL = N_CELL - N_MACRO
# power per RU in mw
p_macro_ru = 200 
p_small_ru = 50
power = np.array([p_macro_ru for i in range(N_MACRO)] + [p_small_ru for i in range(N_SMALL)])


In [105]:
# below is the final solution you got from your program. 
# All of them are vectors, of which the elements are respectively for cells, sequentially from 0 to N_CELL-1

solution=pd.read_csv('solution.csv')

noma_load = list(map(float,solution['noma_load']))
power_allocation = list(map(literal_eval,solution['power_allocation']))
resource_allocation_oma = list(map(literal_eval,solution['resource_allocation_oma']))
resource_allocation_noma = list(map(literal_eval,solution['resource_allocation_noma']))
pair_selection = list(map(literal_eval,solution['pair_selection']))

In [106]:
df = pd.DataFrame()
for j in ue_list:
    d_j = demand_list[j]
    
    i = cell_of_ues[j]
    
    x_j = resource_allocation_oma[i][j]
    d_j_oma = demand_oma(j, x_j, noma_load)
    
    h = -1
    for pair in pair_selection[i]:
        if j in pair:
            h = pair[0] if pair[1]==j else pair[1]
    if h>=0:
        x_u = resource_allocation_noma[i][pair]
        if j==pair[0]:
            q_j, q_h = power_allocation[i][pair]
            d_j_noma = demand_noma(pair[0], pair[1], q_j, q_h, x_u, noma_load)
        else:
            q_h, q_j = power_allocation[i][pair]
            d_j_noma = demand_noma(pair[1], pair[0], q_j, q_h, x_u, noma_load)
    else:
        d_j_noma = 0
    df = df.append({'UE': j, 'd_oma': d_j_oma, 'd_noma': d_j_noma, 'd': d_j}, ignore_index=True)
    
df = df[['UE', 'd_oma', 'd_noma', 'd']].sort_values(by='UE')

df.to_csv('table.csv', index=False)
print(df)

        UE         d_oma  d_noma       d
0      0.0  0.000000e+00  0.0967  0.0967
1      1.0  0.000000e+00  0.0967  0.0967
2      2.0  0.000000e+00  0.0967  0.0967
3      3.0  2.721158e-08  0.0967  0.0967
4      4.0  4.658306e-08  0.0967  0.0967
..     ...           ...     ...     ...
205  205.0  6.744687e-10  0.0967  0.0967
206  206.0  2.781899e-08  0.0967  0.0967
207  207.0  0.000000e+00  0.0967  0.0967
208  208.0  2.540643e-09  0.0967  0.0967
209  209.0  1.384295e-08  0.0967  0.0967

[210 rows x 4 columns]
